# Breakfast-DASH notebook for interactive visualization and experimentation

I am going to copy sample code from dash documentation to create a [minimal dash app](https://dash.plotly.com/minimal-app) and then look at the code and visualization to get a jump start on learning dash. At this point I have read through some of the documentation and watched a few videos on dash and feel confident that I can get something to work fairly quickly.

In [1]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder_unfiltered.csv')

app = Dash(__name__)

app.layout = html.Div([
    html.H1(children='Title of Dash App', style={'textAlign':'center'}),
    dcc.Dropdown(df.country.unique(), 'Canada', id='dropdown-selection'),
    dcc.Graph(id='graph-content')
])

@callback(
    Output('graph-content', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_graph(value):
    dff = df[df.country==value]
    return px.line(dff, x='year', y='pop')

if __name__ == '__main__':
    app.run(debug= True)


Alright that wasn't too difficult. My main setup was creating the virtual environment. This can be done with the following commands. These instructions are specific to my machine (see Github README for machine/OS/python versions); results may vary

1. python3.10 -m pip install --user virtualenv
2. python3.10 -m venv BreakfastDashEnv 
3. source venv/bin/activate
4. Install libraries using [instructions from the dash documentation](https://dash.plotly.com/installation)

<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>TODO</strong>
    <p>Add venv instructions to the README</p>
</div>

First I will go over what I first noticed and then try to understand the code in more depth.

1. Graph looks pretty nice at first glance. 
2. In dark mode, the chart title was very difficult to read as the title was black text on a dark gray background. This is more of an advanced issue, but changing the CSS could either be dynamic or strictly enforce a dark or light mode. I will use a light mode because their might be less issues with visibility.
2. I would adjust the y-axis scale to start at the minimum value (or better round down to the nearest million so 14M). At a quick glance the user might think the minimum value is 15M when it is not. Not too bad in this example, but I have this issue look worse. Overall decent scaling
3. Data points are interactive. Good that they are hidden as to not distract the suer from the main trend of populations of countries over time.
4. I like that you can zoom in if needed and also that you can select an input to change the line chart.
5. I can see how higher up personnel may want very customized dashboards. This could lead to more time used on a minority of features.
6. The graph semi-scales well at different browser sizes. Wonder how this could work on different form factors/devices.

Now I can go through the code to understand dash better. This will be broken down by line number ranges:

1-3: Standard importants for dash/plotly as well as pandas which I am familiar with. Pandas is properly used to convert any data sources to dataframes and do processing on the data easily.

5: Read the data from csv file and convert into a pandas df. The data has the following headers and samples values. EDA will not be done on this data, but would normally be done.

In [2]:
df.head()

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
1,Afghanistan,Asia,1957,30.332,9240934,820.853030
2,Afghanistan,Asia,1962,31.997,10267083,853.100710
3,Afghanistan,Asia,1967,34.020,11537966,836.197138
4,Afghanistan,Asia,1972,36.088,13079460,739.981106


7: Seems like standard boilerplate code to make sure dash look for files/resources in the correct locations. It creates a dash instance to start the dash app.

9-13: High level HTML abstraction. I have a small amount of web dev experience and am looking forward to learning more in a web development class I am taking next semester for my Master'd degree in Computer Science. This feels like pythonic HTML, which sounds weird but is intuitive enough. It seems like a header is made with the cetnered title of the dashboard. Then the dropdown is added with all of the unique countries in the df. The default selection is Canada. An id is assigned; this might be if more elements are created, then the elements can be modified by their id. Then the graph element is added to the higher level div, but nothing is done with it yet. I wonder how other styles or if more elements are added, how will the dashboard look. Is CSS needed Will the elements append to the end of the dashboard or will the dashboard flex the html elements.

15-18: This seems like some way in dash/python to add interactiveness to the dashboard. This will be abstracted for now, but I am curious how this effects the code. Will not look into now. I will just assume this works automagically. It somehow changes the figure and value variables.

19-21: The value is the drop down selection and is accessed in the df. The years and population for all the rows with the specific country selected are plotted using some plotly library.

23-24: Run the app in debug mode. It seem like debug mode might just add the <> interactive button with basics network and server stats. 

Overall I like the dash framework. It feels like a more interactive matplotlib with html/css/JS touches for charts and viz that are ready to be put on a website. With my pandas experience, it feels natural to quickly get started. I will try to finish my other issues/tickets in a python file (not notebook). But first I want to modify the code to understand it better.

I am going to change the Title of the dashbaord, align the title left and change the color to purple. Change the default country to United States, make 2 dropdowns and charts side by side, and run where debug is false.

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder_unfiltered.csv')

app = Dash(__name__)

app.layout = html.Div([
    html.H1(children='Country Populations over Time', style={'textAlign': 'left', 'color': 'purple'}),
    html.Div([
        dcc.Dropdown(options=[{'label': country, 'value': country} for country in df.country.unique()], value='United States', id='dropdown-selection1'),
        dcc.Graph(id='graph-content1')], style={'display': 'inline-block', 'width': '50%'}),
    html.Div([
        dcc.Dropdown(options=[{'label': country, 'value': country} for country in df.country.unique()], value='United States', id='dropdown-selection2'),
        dcc.Graph(id='graph-content2')], style={'display': 'inline-block', 'width': '50%'})
])

@app.callback(
    Output('graph-content1', 'figure'),
    Output('graph-content2', 'figure'),
    Input('dropdown-selection1', 'value'),
    Input('dropdown-selection2', 'value')
)
def update_graph(value1, value2):
    dff1 = df[df.country == value1]
    dff2 = df[df.country == value2]
    fig1 = px.line(dff1, x='year', y='pop')
    fig2 = px.line(dff2, x='year', y='pop')
    return fig1, fig2

if __name__ == '__main__':
    app.run(debug= False)


Changes are as expected! Nice. I think the inline CSS could be problematic for large dashboards. A separate .css file might be a better practice. On to the other tasks.

Normally in matlab and matplotlib, you would have to deal with oddly behaving subplots to put multiple charts in a single chart. I know there are annoying problems aligning elements more dynamically (use flexbox or other software), but this felt a little more natural for a quick implementation.

<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>TODO</strong>
    <p>One issue with jupyter notebooks and dash is that the first app is still running, so when the second app is ran, the first app updates with 2 charts. Minor issue that can be resolved later. May not be an issue if each server is running its own app or some work around is possible. It also seems like this occurs when exporting jupyter notebook to html. Can make an issue.</p>
</div>